# Import packages

In [1]:
import os
import sys
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

Using TensorFlow backend.
/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pa/.virtualenvs/groover/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pa/.virtualenvs/groover/lib/p

In [32]:
# Standard packages
import pandas as pd
import numpy as np
import pickle

# Sklearn
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

# LGBM
from lightgbm import LGBMRegressor

# Keras
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
from IPython.display import SVG

# Plotly
import plotly.express as px

# Load data

In [6]:
sub = pd.read_csv('./data/raw/submission_history.csv')

# Load embeddings

In [4]:
embeddings_index = {}
with open('./cc.fr.300.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 2000000 word vectors.


In [35]:
texts = sub.track_info.fillna('Inconnu').values

MAX_NUM_WORDS = max([len(text) for text in texts])
MAX_SEQUENCE_LENGTH  = 1000

In [36]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 38848 unique tokens.


In [37]:
EMBEDDING_DIM = 300

In [38]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [39]:
N_FOLDS = 5
RANDOM_SEED = 42

dataset = pd.read_csv('./data/preprocessed/merged_dataset.csv')
X = dataset.drop(columns=['id', 'track_id', 'band_id', 'influencer_id', 'score'])
y = dataset.score
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

In [40]:
band_data = X.filter(regex='_band')
influencer_data = X.filter(regex='_influencer')

influencer_kind = X.influencer_kind
i_kind_idx = X.columns.get_loc('influencer_kind')

b_data_idx = [X.columns.get_loc(c) for c in band_data]
i_data_idx = [X.columns.get_loc(c) for c in influencer_data]

In [41]:
def build_NN_model(
    i_emb_dim=16, 
    b_emb_dim=16, 
    kind_emb_dim=8, 
    last_dense=16, 
    dropout=0.2,
    activation='relu'
):
    """
    Build simple MLP Neural Network.
    """
    # Influencer embedding
    influencer_input = Input(shape=[influencer_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation=activation, name="Influencer-Embedding")(influencer_input)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(
        influencer_kind.nunique(), 
        kind_emb_dim, 
        name="Influencer-Kind-Embedding"
    )(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(name="Influencer-Full-Embedding", axis=-1)(
        [influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)]
    )
    
    # Band embedding
    band_input = Input(shape=[band_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation=activation, name="Band-Embedding")(band_input)
    
    # Track embedding
    embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Conv1D(64, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Dropout(rate=dropout)(x)
    x = Conv1D(32, 5, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(rate=dropout)(x)
    seq_emb = Dense(8, activation='relu')(x)
    
    # Concatenate and create product
    concat = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding, seq_emb])
    dense = Dense(last_dense, activation=activation, name="Dense")(concat)
    
    # Dropout
    dropout = Dropout(rate=dropout)(dense)
    
    # Output
    output = Dense(1, activation=activation, name="Output")(dropout)
    band_embedding_model = Model([influencer_input, band_input, influencer_kind_input, sequence_input],
                                 band_embedding)
    influencer_embedding_model = Model([influencer_input, band_input, influencer_kind_input, sequence_input],
                                       influencer_full_emb)
    model = Model([influencer_input, band_input, influencer_kind_input, sequence_input], output)
    model.compile('adam', 'mean_squared_error')
    
    return model, band_embedding_model, influencer_embedding_model

In [42]:
EPOCHS = 200
PATIENCE = 20
BATCH_SIZE = 32

nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    docs_train, docs_test = data[train_index], data[test_index]

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    docs_train, docs_valid = docs_train[tridx], docs_train[vidx]
    
    # Build model
    model, band_embedding_model, influencer_embedding_model = build_NN_model(dropout=0.2)
    
    # Early stopping callback
    es = EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                  patience=5)
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, i_kind_idx], docs_train], 
              y_train,
              validation_data=([X_valid[:, i_data_idx], 
                                X_valid[:, b_data_idx], 
                                X_valid[:, i_kind_idx],
                                docs_valid], 
                               y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es, reduce_lr],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, i_kind_idx], docs_test]), 
        y_test
    ))

nn_score /= N_FOLDS

Train on 60267 samples, validate on 6697 samples
Epoch 1/200
60267/60267 [==============================] - 47s 783us/step - loss: 0.1275 - val_loss: 0.1073
Epoch 2/200
60267/60267 [==============================] - 47s 778us/step - loss: 0.1085 - val_loss: 0.1048
Epoch 3/200
60267/60267 [==============================] - 45s 749us/step - loss: 0.1054 - val_loss: 0.0996
Epoch 4/200
60267/60267 [==============================] - 44s 729us/step - loss: 0.1025 - val_loss: 0.1001
Epoch 5/200
60267/60267 [==============================] - 44s 723us/step - loss: 0.1011 - val_loss: 0.0965
Epoch 6/200
60267/60267 [==============================] - 44s 732us/step - loss: 0.1005 - val_loss: 0.0973
Epoch 7/200
60267/60267 [==============================] - 44s 733us/step - loss: 0.0996 - val_loss: 0.0966
Epoch 8/200
60267/60267 [==============================] - 46s 756us/step - loss: 0.0992 - val_loss: 0.0978
Epoch 9/200
60267/60267 [==============================] - 46s 767us/step - loss: 0.098

Epoch 76/200
60267/60267 [==============================] - 45s 753us/step - loss: 0.0872 - val_loss: 0.0923
Epoch 77/200
60267/60267 [==============================] - 45s 750us/step - loss: 0.0868 - val_loss: 0.0924
Epoch 78/200
60267/60267 [==============================] - 46s 756us/step - loss: 0.0870 - val_loss: 0.0923
Epoch 79/200
60267/60267 [==============================] - 46s 757us/step - loss: 0.0875 - val_loss: 0.0923
Epoch 80/200
60267/60267 [==============================] - 46s 757us/step - loss: 0.0874 - val_loss: 0.0924
Epoch 81/200
60267/60267 [==============================] - 46s 758us/step - loss: 0.0871 - val_loss: 0.0923
Epoch 82/200
60267/60267 [==============================] - 45s 753us/step - loss: 0.0870 - val_loss: 0.0923
Epoch 83/200
60267/60267 [==============================] - 46s 758us/step - loss: 0.0868 - val_loss: 0.0924
Restoring model weights from the end of the best epoch
Epoch 00083: early stopping
Train on 60268 samples, validate on 6697 samp

60268/60268 [==============================] - 45s 753us/step - loss: 0.0893 - val_loss: 0.0946
Epoch 68/200
60268/60268 [==============================] - 45s 746us/step - loss: 0.0895 - val_loss: 0.0946
Epoch 69/200
60268/60268 [==============================] - 46s 760us/step - loss: 0.0894 - val_loss: 0.0946
Epoch 70/200
60268/60268 [==============================] - 46s 756us/step - loss: 0.0895 - val_loss: 0.0946
Epoch 71/200
60268/60268 [==============================] - 46s 760us/step - loss: 0.0894 - val_loss: 0.0946
Epoch 72/200
60268/60268 [==============================] - 46s 766us/step - loss: 0.0893 - val_loss: 0.0946
Epoch 73/200
60268/60268 [==============================] - 45s 750us/step - loss: 0.0890 - val_loss: 0.0946
Epoch 74/200
60268/60268 [==============================] - 45s 745us/step - loss: 0.0893 - val_loss: 0.0946
Epoch 75/200
60268/60268 [==============================] - 45s 754us/step - loss: 0.0894 - val_loss: 0.0946
Epoch 76/200
60268/60268 [======

60268/60268 [==============================] - 46s 760us/step - loss: 0.0879 - val_loss: 0.0929
Epoch 56/200
60268/60268 [==============================] - 46s 759us/step - loss: 0.0876 - val_loss: 0.0929
Restoring model weights from the end of the best epoch
Epoch 00056: early stopping
Train on 60268 samples, validate on 6697 samples
Epoch 1/200
60268/60268 [==============================] - 48s 796us/step - loss: 0.1266 - val_loss: 0.1041
Epoch 2/200
60268/60268 [==============================] - 46s 760us/step - loss: 0.1084 - val_loss: 0.0980
Epoch 3/200
60268/60268 [==============================] - 46s 762us/step - loss: 0.1037 - val_loss: 0.0967
Epoch 4/200
60268/60268 [==============================] - 46s 761us/step - loss: 0.1020 - val_loss: 0.0960
Epoch 5/200
60268/60268 [==============================] - 46s 763us/step - loss: 0.1007 - val_loss: 0.0955
Epoch 6/200
60268/60268 [==============================] - 46s 759us/step - loss: 0.0996 - val_loss: 0.0977
Epoch 7/200
602

60268/60268 [==============================] - 46s 763us/step - loss: 0.0854 - val_loss: 0.0936
Epoch 29/200
60268/60268 [==============================] - 46s 760us/step - loss: 0.0847 - val_loss: 0.0939
Epoch 30/200
60268/60268 [==============================] - 46s 767us/step - loss: 0.0846 - val_loss: 0.0938
Epoch 31/200
60268/60268 [==============================] - 46s 765us/step - loss: 0.0844 - val_loss: 0.0938
Epoch 32/200
60268/60268 [==============================] - 46s 761us/step - loss: 0.0843 - val_loss: 0.0939
Epoch 33/200
60268/60268 [==============================] - 46s 764us/step - loss: 0.0845 - val_loss: 0.0937
Epoch 34/200
60268/60268 [==============================] - 46s 763us/step - loss: 0.0840 - val_loss: 0.0939
Epoch 35/200
60268/60268 [==============================] - 47s 775us/step - loss: 0.0839 - val_loss: 0.0940
Epoch 36/200
60268/60268 [==============================] - 46s 766us/step - loss: 0.0840 - val_loss: 0.0942
Epoch 37/200
60268/60268 [======

In [43]:
print(nn_score)

0.3075894541807684


In [44]:
def build_NN_model(
    i_emb_dim=16, 
    b_emb_dim=16, 
    kind_emb_dim=8, 
    last_dense=16, 
    dropout=0.2,
    activation='relu'
):
    """
    Build simple MLP Neural Network.
    """
    # Influencer embedding
    influencer_input = Input(shape=[influencer_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation=activation, name="Influencer-Embedding")(influencer_input)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(
        influencer_kind.nunique(), 
        kind_emb_dim, 
        name="Influencer-Kind-Embedding"
    )(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(name="Influencer-Full-Embedding", axis=-1)(
        [influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)]
    )
    
    # Band embedding
    band_input = Input(shape=[band_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation=activation, name="Band-Embedding")(band_input)
    
    # Track embedding
    embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = Conv1D(64, 5, activation='relu')(embedded_sequences)
    x = MaxPooling1D(5)(x)
    x = Dropout(rate=dropout)(x)
    x = Conv1D(32, 5, activation='relu')(x)
    x = Dropout(rate=dropout)(x)
    x = Conv1D(32, 5, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(rate=dropout)(x)
    seq_emb = Dense(16, activation='relu')(x)
    
    # Concatenate and create product
    concat = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding, seq_emb])
    dense = Dense(last_dense, activation=activation, name="Dense")(concat)
    
    # Dropout
    dropout = Dropout(rate=dropout)(dense)
    
    # Output
    output = Dense(1, activation=activation, name="Output")(dropout)
    band_embedding_model = Model([influencer_input, band_input, influencer_kind_input, sequence_input],
                                 band_embedding)
    influencer_embedding_model = Model([influencer_input, band_input, influencer_kind_input, sequence_input],
                                       influencer_full_emb)
    model = Model([influencer_input, band_input, influencer_kind_input, sequence_input], output)
    model.compile('adam', 'mean_squared_error')
    
    return model, band_embedding_model, influencer_embedding_model

In [45]:
EPOCHS = 200
PATIENCE = 20
BATCH_SIZE = 32

nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    docs_train, docs_test = data[train_index], data[test_index]

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    docs_train, docs_valid = docs_train[tridx], docs_train[vidx]
    
    # Build model
    model, band_embedding_model, influencer_embedding_model = build_NN_model(dropout=0.3)
    
    # Early stopping callback
    es = EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                  patience=5)
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, i_kind_idx], docs_train], 
              y_train,
              validation_data=([X_valid[:, i_data_idx], 
                                X_valid[:, b_data_idx], 
                                X_valid[:, i_kind_idx],
                                docs_valid], 
                               y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es, reduce_lr],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, i_kind_idx], docs_test]), 
        y_test
    ))

nn_score /= N_FOLDS

Train on 60267 samples, validate on 6697 samples
Epoch 1/200
60267/60267 [==============================] - 51s 841us/step - loss: 0.1276 - val_loss: 0.1095
Epoch 2/200
60267/60267 [==============================] - 47s 782us/step - loss: 0.1123 - val_loss: 0.1033
Epoch 3/200
60267/60267 [==============================] - 47s 775us/step - loss: 0.1082 - val_loss: 0.1004
Epoch 4/200
60267/60267 [==============================] - 47s 780us/step - loss: 0.1070 - val_loss: 0.0983
Epoch 5/200
60267/60267 [==============================] - 47s 787us/step - loss: 0.1054 - val_loss: 0.0980
Epoch 6/200
60267/60267 [==============================] - 47s 784us/step - loss: 0.1037 - val_loss: 0.0973
Epoch 7/200
60267/60267 [==============================] - 47s 780us/step - loss: 0.1034 - val_loss: 0.0964
Epoch 8/200
60267/60267 [==============================] - 47s 775us/step - loss: 0.1020 - val_loss: 0.0952
Epoch 9/200
60267/60267 [==============================] - 47s 783us/step - loss: 0.101

60268/60268 [==============================] - 48s 789us/step - loss: 0.1060 - val_loss: 0.0992
Epoch 4/200
60268/60268 [==============================] - 47s 779us/step - loss: 0.1040 - val_loss: 0.0991
Epoch 5/200
60268/60268 [==============================] - 47s 783us/step - loss: 0.1026 - val_loss: 0.1007
Epoch 6/200
60268/60268 [==============================] - 47s 779us/step - loss: 0.1016 - val_loss: 0.0966
Epoch 7/200
60268/60268 [==============================] - 47s 782us/step - loss: 0.1007 - val_loss: 0.0981
Epoch 8/200
60268/60268 [==============================] - 47s 784us/step - loss: 0.0999 - val_loss: 0.0964
Epoch 9/200
60268/60268 [==============================] - 47s 782us/step - loss: 0.0999 - val_loss: 0.0965
Epoch 10/200
60268/60268 [==============================] - 47s 783us/step - loss: 0.0992 - val_loss: 0.0962
Epoch 11/200
60268/60268 [==============================] - 47s 785us/step - loss: 0.0988 - val_loss: 0.0968
Epoch 12/200
60268/60268 [============

60268/60268 [==============================] - 48s 791us/step - loss: 0.0999 - val_loss: 0.0963
Epoch 10/200
60268/60268 [==============================] - 48s 789us/step - loss: 0.0996 - val_loss: 0.0956
Epoch 11/200
60268/60268 [==============================] - 48s 794us/step - loss: 0.0989 - val_loss: 0.0954
Epoch 12/200
60268/60268 [==============================] - 48s 792us/step - loss: 0.0989 - val_loss: 0.0953
Epoch 13/200
60268/60268 [==============================] - 47s 788us/step - loss: 0.0985 - val_loss: 0.0949
Epoch 14/200
60268/60268 [==============================] - 48s 792us/step - loss: 0.0981 - val_loss: 0.0959
Epoch 15/200
60268/60268 [==============================] - 48s 795us/step - loss: 0.0980 - val_loss: 0.0955
Epoch 16/200
60268/60268 [==============================] - 48s 791us/step - loss: 0.0977 - val_loss: 0.0952
Epoch 17/200
60268/60268 [==============================] - 48s 795us/step - loss: 0.0976 - val_loss: 0.0967
Epoch 18/200
60268/60268 [======

60268/60268 [==============================] - 48s 789us/step - loss: 0.0959 - val_loss: 0.0932
Epoch 24/200
60268/60268 [==============================] - 48s 790us/step - loss: 0.0954 - val_loss: 0.0940
Epoch 25/200
60268/60268 [==============================] - 47s 784us/step - loss: 0.0955 - val_loss: 0.0938
Epoch 26/200
60268/60268 [==============================] - 48s 789us/step - loss: 0.0953 - val_loss: 0.0934
Epoch 27/200
60268/60268 [==============================] - 47s 786us/step - loss: 0.0946 - val_loss: 0.0930
Epoch 28/200
60268/60268 [==============================] - 47s 783us/step - loss: 0.0942 - val_loss: 0.0933
Epoch 29/200
60268/60268 [==============================] - 47s 788us/step - loss: 0.0943 - val_loss: 0.0933
Epoch 30/200
60268/60268 [==============================] - 47s 785us/step - loss: 0.0942 - val_loss: 0.0935
Epoch 31/200
60268/60268 [==============================] - 48s 793us/step - loss: 0.0941 - val_loss: 0.0934
Epoch 32/200
60268/60268 [======

60268/60268 [==============================] - 49s 809us/step - loss: 0.0925 - val_loss: 0.0942
Epoch 31/200
60268/60268 [==============================] - 48s 795us/step - loss: 0.0928 - val_loss: 0.0932
Epoch 32/200
60268/60268 [==============================] - 48s 801us/step - loss: 0.0923 - val_loss: 0.0938
Epoch 33/200
60268/60268 [==============================] - 48s 803us/step - loss: 0.0923 - val_loss: 0.0939
Epoch 34/200
60268/60268 [==============================] - 49s 806us/step - loss: 0.0919 - val_loss: 0.0933
Epoch 35/200
60268/60268 [==============================] - 49s 805us/step - loss: 0.0921 - val_loss: 0.0939
Epoch 36/200
60268/60268 [==============================] - 49s 807us/step - loss: 0.0921 - val_loss: 0.0936
Epoch 37/200
60268/60268 [==============================] - 48s 801us/step - loss: 0.0912 - val_loss: 0.0938
Epoch 38/200
60268/60268 [==============================] - 49s 806us/step - loss: 0.0910 - val_loss: 0.0937
Epoch 39/200
60268/60268 [======

In [46]:
print(nn_score)

0.3080378590783084


In [25]:
EPOCHS = 200
PATIENCE = 15
BATCH_SIZE = 32

nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    docs_train, docs_test = data[train_index], data[test_index]

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    docs_train, docs_valid = docs_train[tridx], docs_train[vidx]
    
    # Build model
    model, band_embedding_model, influencer_embedding_model = build_NN_model(dropout=0.1)
    
    # Early stopping callback
    es = EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, i_kind_idx], docs_train], 
              y_train,
              validation_data=([X_valid[:, i_data_idx], 
                                X_valid[:, b_data_idx], 
                                X_valid[:, i_kind_idx],
                                docs_valid], 
                               y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, i_kind_idx], docs_test]), 
        y_test
    ))

nn_score /= N_FOLDS

Train on 60267 samples, validate on 6697 samples
Epoch 1/200
17824/60267 [=======>......................] - ETA: 21s - loss: 0.1418

KeyboardInterrupt: 

In [29]:
print(nn_score)

0.3077258867925123
